## Batch Query

Prior to the games, questions to answer with batch querying:
- Which discipline has the most athletes for each country?
- Provide an overview of the number of athletes from all countries in each discipline and overall
- Which countries have the highest and lowest proportions of athletes to coaches?
- Which discipline has the highest proportion of female technical officials?
- Which discipline has the oldest and youngest athletes on average? Which discipline has the smallest and tallest athletes? 

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

# Setting Spark configuration
sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("Assignment2-BatchQuery")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")
# Create the spark session, which is the entry point to Spark SQL engine
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

# Setup hadoop fs configuration for schema gs://
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

# Google Storage File Path
# Have to collect together data which has been partitioned when saved back into the bucket
athletes_gsc_file_path = 'gs://dejads_output_assignment2_team1/athletes_clean.csv/*.csv' 
coaches_gsc_file_path = 'gs://dejads_output_assignment2_team1/coaches_clean.csv/*.csv' 
tech_offic_gsc_file_path = 'gs://dejads_output_assignment2_team1/tech_offic_clean.csv/*.csv' 
country_file_path = 'gs://dejads_output_assignment2_team1/country.csv/*.csv' 
disciplines_file_path = 'gs://dejads_output_assignment2_team1/disciplines.csv/*.csv' 

# Create data frames, return their schemas and show the first row
athletes_df = spark.read.format("csv").option("header", "true") \
       .load(athletes_gsc_file_path)
athletes_df.printSchema()
athletes_df.show(1)

coaches_df = spark.read.format("csv").option("header", "true") \
       .load(coaches_gsc_file_path)
coaches_df.printSchema()
coaches_df.show(1)

tech_offic_df = spark.read.format("csv").option("header", "true") \
       .load(tech_offic_gsc_file_path)
tech_offic_df.printSchema()
tech_offic_df.show(1)

country_df = spark.read.format("csv").option("header", "true") \
       .load(country_file_path)
country_df.printSchema()
country_df.show(1)

disc_df = spark.read.format("csv").option("header", "true") \
       .load(disciplines_file_path)
disc_df.printSchema()
disc_df.show(1)

root
 |-- name: string (nullable = true)
 |-- short_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- birth_date: string (nullable = true)
 |-- birth_place: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- discipline_code: string (nullable = true)
 |-- residence_place: string (nullable = true)
 |-- residence_country: string (nullable = true)
 |-- url: string (nullable = true)
 |-- height_m: string (nullable = true)
 |-- birth_country_code: string (nullable = true)

+---------------+----------+------+----------+-----------+------------+---------------+---------------+-----------------+--------------------+--------+------------------+
|           name|short_name|gender|birth_date|birth_place|country_code|discipline_code|residence_place|residence_country|                 url|height_m|birth_country_code|
+---------------+----------+------+----------+-----------+------------+---------------+---------------+-----------------+------------------

In [6]:
# Define function to save data to BigQuery
# Use the Cloud Storage bucket for temporary BigQuery export data used by the connector
def saveBigQuery(df,table_name):
    # update personal bucket name
    bucket = "dejads_temp_assignment2_team1"
    spark.conf.set('temporaryGcsBucket', bucket)
    # Setup hadoop fs configuration for schema gs://
    conf = spark.sparkContext._jsc.hadoopConfiguration()
    conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
    conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
    # Saving the data to BigQuery
    # update personal project and table name
    df.write.format('bigquery') \
      .option('table', 'de2021-assignment2.assignment2.' + table_name) \
      .mode("append") \
      .save()

### Which discipline has the most athletes for each country?

In [3]:
from pyspark.sql.functions import max, dense_rank, desc, col
from pyspark.sql.window import Window

# Create dataframe with number of athletes from each country for each discipline
gp_cty_disc_df = athletes_df.groupBy("country_code", "discipline_code").count().withColumnRenamed('count', 'no_athletes')

# Window query, partition by country_code, descending number of athletes
windowSpec = Window\
  .partitionBy("country_code")\
  .orderBy(desc("no_athletes"))

# Dense rank over window query returning only first row (highest number of athletes)
top_gp_cty_disc_df = gp_cty_disc_df.withColumn('dense_rank', dense_rank().over(windowSpec)).where(col('dense_rank') == 1)
# Join to country dataframe to display country name
top_gp_cty_disc_df = top_gp_cty_disc_df.join(country_df, "country_code", "left_outer").drop('country_code')
# Join to discipline dataframe to display discipline
top_gp_cty_disc_df = top_gp_cty_disc_df.join(disc_df, "discipline_code", "left_outer").drop('discipline_code')

In [4]:
# Selecting columns to display and order alphabetically on country and discipline
top_gp_cty_disc_df = top_gp_cty_disc_df.select('country', 'discipline', 'no_athletes').orderBy('country', 'discipline')
top_gp_cty_disc_df.show()

+-------------------+-------------+-----------+
|            country|   discipline|no_athletes|
+-------------------+-------------+-----------+
|        Afghanistan|    Athletics|          2|
|            Albania|    Athletics|          2|
|            Albania|     Swimming|          2|
|            Albania|Weightlifting|          2|
|            Algeria|       Boxing|          8|
|            Algeria|    Wrestling|          8|
|     American Samoa|      Sailing|          2|
|     American Samoa|     Swimming|          2|
|            Andorra|    Athletics|          1|
|            Andorra| Canoe Slalom|          1|
|             Angola|     Handball|         14|
|Antigua and Barbuda|    Athletics|          2|
|Antigua and Barbuda|     Swimming|          2|
|          Argentina|       Hockey|         36|
|            Armenia|    Wrestling|          6|
|              Aruba|     Swimming|          2|
|          Australia|    Athletics|         63|
|            Austria|    Athletics|     

In [7]:
# Save result into serving layer in BigQuery
saveBigQuery(top_gp_cty_disc_df,'top_disc_per_country')

### Provide an overview of the number of athletes from all countries in each discipline and overall

In [8]:
from pyspark.sql.functions import count

# Cube on athletes for country and discipline to give the total number of athletes per country and number per country for each discipline
ath_cube = athletes_df.cube("country_code", "discipline_code").agg(count(col("name")))\
    .withColumnRenamed('count(name)', 'no_athletes')

# Join to country dataframe to display country name
ath_cube = ath_cube.join(country_df, "country_code", "left_outer").drop('country_code')
# Join to discipline dataframe to display discipline
ath_cube = ath_cube.join(disc_df, "discipline_code", "left_outer").drop('discipline_code')

In [9]:
# Filling nulls from cube with total for clarity (there are no nulls for country and discipline in original dataset)
ath_cube = ath_cube.select('country', 'discipline', 'no_athletes').na.fill("Total").orderBy("country", "discipline")
ath_cube.show()

+-----------+-------------------+-----------+
|    country|         discipline|no_athletes|
+-----------+-------------------+-----------+
|Afghanistan|          Athletics|          2|
|Afghanistan|           Shooting|          1|
|Afghanistan|           Swimming|          1|
|Afghanistan|          Taekwondo|          1|
|Afghanistan|              Total|          5|
|    Albania|Artistic Gymnastics|          1|
|    Albania|          Athletics|          2|
|    Albania|               Judo|          1|
|    Albania|           Shooting|          1|
|    Albania|           Swimming|          2|
|    Albania|              Total|          9|
|    Albania|      Weightlifting|          2|
|    Algeria|          Athletics|          5|
|    Algeria|             Boxing|          8|
|    Algeria|       Canoe Sprint|          1|
|    Algeria|       Cycling Road|          2|
|    Algeria|            Fencing|          4|
|    Algeria|               Judo|          2|
|    Algeria|             Karate| 

In [10]:
# Save result into serving layer in BigQuery
saveBigQuery(ath_cube,'overview_athletes_disc_per_country')

### Which countries have the highest and lowest proportion of athletes to coaches? (top/bottom 3)

In [11]:
# Grouping athletes by country code with count of athletes per country
gp_ath_cty_df = athletes_df.groupBy("country_code").count().withColumnRenamed('count', 'no_athletes')
# Grouping coaches by country code with count of coaches per country
gp_coach_cty_df = coaches_df.groupBy("country_code").count().withColumnRenamed('count', 'no_coaches')

# Joining athletes table to coaches table- using full outer join in case there is a country in one table not present in the other
ath_coach_cty = gp_ath_cty_df.join(gp_coach_cty_df, 'country_code', 'full_outer')
# Join to country dataframe to display country name
ath_coach_cty = ath_coach_cty.join(country_df, 'country_code', 'inner')

In [12]:
from pyspark.sql.functions import round, desc, asc

# Selecting country, number of coaches, number of athletes
# Replace any missing data with 0
# Creating new column with the proportion of athletes to coaches
# If any countries with no coaches then fill in proportion column with infinity
ath_coach_cty = ath_coach_cty.select('country', 'no_coaches', 'no_athletes') \
    .na.fill(0) \
    .withColumn('proportionAthleteToCoach', col('no_athletes')/col('no_coaches')) \
    .na.fill(float('inf')) \

# Create window function with the proportion ascending and descending
windowSpecDesc = Window\
  .orderBy(col("proportionAthleteToCoach").desc())
windowSpecAsc = Window\
  .orderBy(col("proportionAthleteToCoach").asc())

# Adding rank columns ascending and descending and filtering to top 3 and bottom 3 rankings
ath_coach_cty = ath_coach_cty.withColumn('desc_rank', dense_rank().over(windowSpecDesc))\
    .withColumn('asc_rank', dense_rank().over(windowSpecAsc)).where((col('desc_rank') <= 3) | (col('asc_rank') <= 3))

# Rounding proportion column to two decimal places for clarity
ath_coach_cty = ath_coach_cty.withColumn('proportionAthleteToCoach', round(col('proportionAthleteToCoach'),2)).orderBy(asc('proportionAthleteToCoach'))
ath_coach_cty.show()

+--------------------+----------+-----------+------------------------+---------+--------+
|             country|no_coaches|no_athletes|proportionAthleteToCoach|desc_rank|asc_rank|
+--------------------+----------+-----------+------------------------+---------+--------+
|           Venezuela|        10|         43|                     4.3|       59|       1|
|       Liechtenstein|         1|          5|                     5.0|       58|       2|
|             Nigeria|         9|         53|                    5.89|       57|       3|
|             Belarus|         1|        103|                   103.0|        3|      57|
|              Turkey|         1|        107|                   107.0|        2|      58|
|        Burkina Faso|         0|          7|                Infinity|        1|      59|
|             Armenia|         0|         17|                Infinity|        1|      59|
|             Jamaica|         0|         61|                Infinity|        1|      59|
|         

In [13]:
# Save result into serving layer in BigQuery
saveBigQuery(ath_coach_cty,'prop_coach_athlete_cty')

### Which discipline has the highest and lowest proportion of female technical officials? (top/ bottom 3)

Note: there are a number of disciplines which do not have technical officials stored in the database. Therefore here we focus on sports which do have technical officials stored.

In [14]:
from pyspark.sql.functions import when, concat, lit

# Rollup of tech officials based on discipline code and gender, then filtering to rows with Females or total number
# Doing join between two table to get row with count of females and total count for each discipline
# Filling any nulls with 0 (i.e. if discipline had no women)
tech_rollup = tech_offic_df.rollup("discipline_code", "gender").count().where((col('gender') == 'Female') | (col('gender').isNull()))
tech_rollup_total = tech_rollup.where(col('gender').isNull() & (~col('discipline_code').isNull()))
tech_rollup_female = tech_rollup.where(col('gender') == 'Female').withColumnRenamed('count', 'female_officials_count').drop('gender')
tech_rollup = tech_rollup_female.join(tech_rollup_total.drop('gender').withColumnRenamed('count', 'count_per_discipline'), \
                                      'discipline_code', how='right_outer').na.fill(0)

# Join technical officials to all disciplines and dropping discipline code
# Using inner join because concentrating on disciplines in dataset
female_df = tech_rollup.join(disc_df, "discipline_code", "inner").drop('discipline_code').na.fill(0)

# Creating percentage column with number of female officials divided by the number of officials in total
female_df = female_df.withColumn('percent_female', col('female_officials_count')/col('count_per_discipline'))

# Created window functions ordering the disciplines on their percentage of female officials
windowSpecAsc = Window\
  .orderBy(col("percent_female").asc())
windowSpecDesc = Window\
  .orderBy(col("percent_female").desc())

# Filtering disciplines based on their ranking of percentage of female officials (top/ bottom three)
female_df = female_df.withColumn('asc_rank', dense_rank().over(windowSpecAsc)) \
    .withColumn('desc_rank', dense_rank().over(windowSpecDesc)).where((col('desc_rank') <= 3) | (col('asc_rank') <= 3))

# Adding column naming if they are top or bottom 3 based on ranking columns
female_df = female_df.withColumn('ranking', when(col('asc_rank') <= 3, 'Bottom 3').when(col('desc_rank') <= 3, 'Top 3').otherwise(None))

# Changing decimal column to actual percentage and choosing columns to display
female_df = female_df.withColumn('percent_female', concat((col('percent_female') * 100).cast("int"),lit('%'))) \
    .select('ranking','discipline', 'female_officials_count','count_per_discipline','percent_female')
female_df.show()

+--------+----------------+----------------------+--------------------+--------------+
| ranking|      discipline|female_officials_count|count_per_discipline|percent_female|
+--------+----------------+----------------------+--------------------+--------------+
|   Top 3|          Hockey|                    14|                  28|           50%|
|   Top 3|      Equestrian|                    17|                  38|           44%|
|   Top 3|      Water Polo|                    40|                 107|           37%|
|Bottom 3|        Handball|                    12|                  74|           16%|
|Bottom 3|Beach Volleyball|                    11|                  72|           15%|
|Bottom 3|       Wrestling|                     7|                  82|            8%|
+--------+----------------+----------------------+--------------------+--------------+



In [15]:
saveBigQuery(female_df,'prop_female_tech_disc')

### Which discipline has the oldest and youngest athletes on average? Which discipline has the smallest and tallest athletes? 

In [16]:
from pyspark.sql.types import DateType, DoubleType
from pyspark.sql.functions import months_between, current_date, round

# Casting height to a DoubleType
athletes_df = athletes_df.withColumn('height_m', col('height_m').cast(DoubleType()))
# Casting birthdate to a DateType 
athletes_df = athletes_df.withColumn('birth_date', col('birth_date').cast(DateType()))
# Calculating age based on number of months between birthdate and now divided by 12 and rounded to two decimal places
athletes_df = athletes_df \
  .withColumn("age_rounded",round(months_between(current_date(),col("birth_date"))/lit(12),2))

The below is carried out with SQL to show other techniques possible with Spark.

In [17]:
# Creating views of tables for SQL
athletes_df.createOrReplaceTempView("dfAthletes")
disc_df.createOrReplaceTempView("dfDisc")

In [18]:
# Creating function which can be adapted per column to find biggest/ smallest discipline on average for that column
def return_top_bottom_avg_discipline(characteristic):
    query = "WITH Q AS (SELECT discipline, ROUND(avg(" + characteristic + "),2) AS avg_" + characteristic + " FROM dfAthletes \
    INNER JOIN dfDisc ON dfDisc.discipline_code = dfAthletes.discipline_code \
    GROUP BY discipline), \
    QA AS (SELECT * FROM Q WHERE avg_" + characteristic + " IS NOT NULL ORDER BY avg_" + characteristic + " DESC LIMIT 1), \
    QD AS (SELECT * FROM Q WHERE avg_" + characteristic + " IS NOT NULL ORDER BY avg_" + characteristic + " ASC LIMIT 1) \
    SELECT * FROM QD UNION SELECT * FROM QA"
    return spark.sql(query)

In [19]:
# Finding youngest and oldest average disciplines
young_old_df = return_top_bottom_avg_discipline('age_rounded')
young_old_df.show()

+-------------------+---------------+
|         discipline|avg_age_rounded|
+-------------------+---------------+
|Rhythmic Gymnastics|          21.41|
|         Equestrian|          39.07|
+-------------------+---------------+



In [20]:
saveBigQuery(young_old_df,'young_old_disc')

In [21]:
# Finding smallest and tallest average disciplines
height_df = return_top_bottom_avg_discipline('height_m')
height_df.show()

+----------+------------+
|discipline|avg_height_m|
+----------+------------+
|Basketball|        1.91|
|Equestrian|        1.57|
+----------+------------+



In [22]:
saveBigQuery(height_df,'short_tall_disc')

In [23]:
# Stop the spark context
spark.stop()